This Notebook was made in Google Colab. So it don't have the virtual environment libraries, local files path and loggings.
Libraries has been installed externally and file paths are given by inputs.

In [ ]:
!apt-get install inxi &> /dev/null
!inxi -C

CPU:
  Topology: Single Core model: Intel Xeon bits: 64 type: MT 
  L2 cache: 55.0 MiB 
  Speed: 2200 MHz min/max: N/A Core speeds (MHz): 1: 2200 2: 2200 


In [ ]:
!lscpu |grep 'Model name'

Model name:                      Intel(R) Xeon(R) CPU @ 2.20GHz


In [ ]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
# installing libraries
!pip install ultralytics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.1/629.1 kB 11.2 MB/s eta 0:00:00


In [ ]:
!pip install python-box

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 34.6 MB/s eta 0:00:00


In [ ]:
from box import ConfigBox
from box.exceptions import BoxValueError
from dataclasses import dataclass
from pathlib import Path
import yaml
import os

In [ ]:
## helper functions


def read_yaml(path_to_yaml: Path) -> ConfigBox:
    """reads yaml file and returns
    Args:
        path_to_yaml (str): path like input
    Raises:
        ValueError: if yaml file is empty
        e: empty file
    Returns:
        ConfigBox: ConfigBox type
    """
    try:
        with open(path_to_yaml) as yaml_file:
            content = yaml.safe_load(yaml_file)
            return ConfigBox(content)
    except BoxValueError:
        raise ValueError("yaml file is empty")
    except Exception as e:
        raise e

In [ ]:
@dataclass(frozen = True)
class TrainModelConfig:
  base_model: Path
  data_file: Path
  saved_model_dir: Path
  epochs: int
  batch_size: int
  image_size: int
  optimizer: str


In [ ]:
CONFIG_FILE_PATH = "/content/drive/MyDrive/Object_Detection_Projects/config.yaml"
PARAMS_FILE_PATH = "/content/drive/MyDrive/Object_Detection_Projects/params.yaml"

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    def get_train_model_config(self) -> TrainModelConfig:

      base_model_path = os.path.join(self.config.base_model.base_model_dir,
                                     self.params.BASE_MODEL.MODEL_NAME)
      config = self.config.train_model
      params = self.params.TRAIN_MODEL_PARAMS

      model_training_config = TrainModelConfig(
          base_model = Path(base_model_path),
          data_file = config.data_file_path,
          saved_model_dir = config.saved_model_dir,
          epochs = params.EPOCHS,
          batch_size= params.BATCH_SIZE,
          image_size = params.IMAGE_SIZE,
          optimizer = params.OPTIMIZER )

      return model_training_config

In [ ]:
from ultralytics import YOLO
import torch

In [ ]:
class TrainModel:
  def __init__(self, config: TrainModelConfig):
    self.config = config

  def train_model(self):
    try:
      model = YOLO(self.config.base_model)
      device = 0 if torch.cuda.is_available() else "cpu"
      model.train(
          data = self.config.data_file,
          imgsz = self.config.image_size,
          epochs = self.config.epochs,
          batch = self.config.batch_size,
          project = self.config.saved_model_dir,
          device = device,
          optimizer= self.config.optimizer
      )
    except Exception as e:
      raise e


In [ ]:
try:
  config = ConfigurationManager()
  train_model_config = config.get_train_model_config()
  model_training = TrainModel(train_model_config)
  model_training.train_model()

except Exception as e:
  raise e